# Курсовая работа

Датасет [Visual Question Answering (VQA)](https://paperswithcode.com/dataset/visual-question-answering)

## Описание

Датасет Visual Question Answering (VQA) из библиотеки Facebook AI Research ParlAI представляет собой коллекцию данных, предназначенных для разработки и тестирования моделей, которые могут интерпретировать визуальные данные и отвечать на вопросы, связанные с этими данными.

Каждый образец в датасете содержит изображение, вопрос на естественном языке, сформулированный относительно этого изображения, и один или несколько возможных ответов, отражающих человеческое восприятие. Вопросы охватывают широкий спектр сложностей, начиная от простых запросов, таких как «Какого цвета машина?», до более сложных, например, «Сколько людей на переднем плане и чем они занимаются?».

Изображения в датасете могут быть из реального мира или сгенерированы, включать сцены с разнообразными объектами, людьми, действиями и контекстами. Визуальная информация сочетается с текстовыми компонентами, чтобы побудить модель понимать не только изолированные аспекты данных, но и их взаимосвязь.

In [ ]:
from parlai.core.params import ParlaiParser
from parlai.core.worlds import create_task
from parlai.core.agents import create_agent
from parlai.utils.logging import logging

# Создаем парсер для параметров
parser = ParlaiParser()
parser.set_defaults(
    task='vqa_v2',  # Указываем название задачи VQA v2
    datatype='train:stream',  # Тип данных (например, обучение, тест)
    batchsize=1,  # Размер батча (для примера установим 1)
)

# Получаем аргументы
opt = parser.parse_args(print_args=False)

# Создаем агент (например, dummy-агент для просмотра данных)
agent = create_agent(opt)
world = create_task(opt, agent)

# Выводим пример из датасета
print("Пример из датасета Visual Question Answering:")
for _ in range(1):  # Загружаем один пример
    world.parley()
    message = world.get_acts()  # Получаем сообщение из мира
    for msg in message:
        if 'image' in msg:
            print("Изображение: доступно (бинарные данные)")
        if 'text' in msg:
            print(f"Вопрос: {msg['text']}")
        if 'labels' in msg:
            print(f"Ответы: {msg['labels']}")

## Статистики

In [ ]:
# Переменные для сбора статистики
num_examples = 0
question_lengths = []
answer_counts = Counter()
question_types = Counter()

print("Сбор статистики по датасету VQA начат...")
for _ in range(10000):  # Ограничиваем до 10,000 примеров для анализа
    world.parley()
    message = world.get_acts()[0]  # Получаем первое сообщение из мира
    if 'text' in message:
        question = message['text']
        question_lengths.append(len(question.split()))
        question_types[question.split()[0].lower()] += 1  # Первая слово как тип вопроса
    if 'labels' in message:
        for label in message['labels']:
            answer_counts[label.lower()] += 1
    num_examples += 1

print(f"Количество примеров: {num_examples}")
print(f"Средняя длина вопроса: {sum(question_lengths) / len(question_lengths):.2f}")
print(f"Топ-10 наиболее частых ответов: {answer_counts.most_common(10)}")
print(f"Топ-10 начальных слов вопросов: {question_types.most_common(10)}")

# Визуализация длины вопросов
plt.figure(figsize=(10, 6))
plt.hist(question_lengths, bins=range(1, 21), color='skyblue', edgecolor='black')
plt.title("Распределение длин вопросов")
plt.xlabel("Длина вопроса (количество слов)")
plt.ylabel("Количество вопросов")
plt.show()

# Визуализация топ-10 ответов
top_answers = answer_counts.most_common(10)
answers, counts = zip(*top_answers)
plt.figure(figsize=(10, 6))
plt.bar(answers, counts, color='salmon')
plt.title("Топ-10 наиболее частых ответов")
plt.xlabel("Ответ")
plt.ylabel("Частота")
plt.xticks(rotation=45)
plt.show()

# Визуализация топ-10 типов вопросов
top_question_types = question_types.most_common(10)
q_types, q_counts = zip(*top_question_types)
plt.figure(figsize=(10, 6))
plt.bar(q_types, q_counts, color='lightgreen')
plt.title("Топ-10 начальных слов вопросов (типов вопросов)")
plt.xlabel("Тип вопроса (начальное слово)")
plt.ylabel("Частота")
plt.xticks(rotation=45)
plt.show()

## Кластеризация/понижение размерности

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# Преобразование текстовых данных в числовые векторы с помощью TF-IDF
vectorizer = TfidfVectorizer(max_features=1000, stop_words='english')
X_tfidf = vectorizer.fit_transform(questions)

# Кластеризация вопросов с использованием KMeans
num_clusters = 10
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
clusters = kmeans.fit_predict(X_tfidf)

# Понижение размерности для визуализации
tsne = TSNE(n_components=2, random_state=42, perplexity=30)
X_tsne = tsne.fit_transform(X_tfidf.toarray())

# Визуализация кластеров
plt.figure(figsize=(12, 8))
for cluster in range(num_clusters):
    cluster_points = X_tsne[clusters == cluster]
    plt.scatter(cluster_points[:, 0], cluster_points[:, 1], label=f"Кластер {cluster}", alpha=0.6)
plt.title("Визуализация кластеров вопросов с помощью T-SNE")
plt.xlabel("Компонента 1")
plt.ylabel("Компонента 2")
plt.legend()
plt.show()

## Нахождение выбросов

In [ ]:
from sklearn.ensemble import IsolationForest
import numpy as np

# Использование Isolation Forest для обнаружения выбросов
iso_forest = IsolationForest(contamination=0.02, random_state=42)  # 2% данных считаются выбросами
outlier_labels = iso_forest.fit_predict(X_tfidf)

# Анализ выбросов
outlier_indices = np.where(outlier_labels == -1)[0]  # Индексы выбросов
print(f"Найдено выбросов: {len(outlier_indices)}")

# Вывод примеров выбросов
print("\nПримеры вопросов-выбросов:")
for idx in outlier_indices[:10]:  # Покажем первые 10 выбросов
    print(f"- {questions[idx]}")

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_tfidf.toarray())

# Отображение данных с выделением выбросов
plt.figure(figsize=(12, 8))
plt.scatter(X_pca[:, 0], X_pca[:, 1], c='lightblue', label='Нормальные данные', alpha=0.5)
plt.scatter(
    X_pca[outlier_indices, 0],
    X_pca[outlier_indices, 1],
    c='red',
    label='Выбросы',
    alpha=0.8,
)
plt.title("Выбросы в данных (на основе PCA и Isolation Forest)")
plt.xlabel("Компонента 1")
plt.ylabel("Компонента 2")
plt.legend()
plt.show()

## ML-systems design

Датасет Visual Question Answering (VQA) является мощным инструментом для разработки и тестирования моделей, которые комбинируют компьютерное зрение и обработку естественного языка.

---

### **1. Искусственный интеллект и мультимодальные модели**
Датасет VQA используется для обучения и оценки моделей, способных объединять визуальную и текстовую информацию. Он позволяет:
- Разрабатывать мультимодальные архитектуры, такие как CLIP, Flamingo или модели, основанные на трансформерах.
- Оценивать способность моделей извлекать осмысленную информацию из изображений и корректно отвечать на вопросы.
- Тренировать модели для более глубокого семантического понимания объектов, действий, контекстов и отношений между элементами на изображении.

---

### **2. Помощники с искусственным интеллектом**
VQA полезен для создания интерактивных приложений, способных понимать и интерпретировать визуальные сцены:
- **Цифровые помощники**: Например, системы для описания и анализа фотографий, которые могут отвечать на вопросы, такие как «Что изображено на фотографии?» или «Кто на переднем плане?».
- **Роботы и автономные системы**: Робот может использовать VQA для взаимодействия с человеком, понимая команды или вопросы, связанные с окружающей средой.

---

### **3. Доступность для людей с ограниченными возможностями**
VQA поддерживает создание технологий для помощи людям с ограниченным зрением. Примеры:
- **Приложения для слабовидящих**: Могут описывать сцены и отвечать на вопросы о них, например, «Сколько людей на фотографии?» или «Есть ли рядом дорога?».
- **Инклюзивные интерфейсы**: Обеспечивают голосовую поддержку для работы с визуальными элементами.

---

### **4. Рекомендательные системы и поиск**
Датасет используется для разработки систем, которые понимают визуальный контекст и текстовые запросы:
- **Поиск по изображениям**: Улучшение поисковых систем, где запросы могут быть текстовыми, а результаты зависят от анализа изображений.
- **Рекомендации в e-commerce**: Ответы на вопросы о продуктах, таких как «Какого цвета эта футболка?» или «Подходит ли эта сумка к черному платью?».

---

### **5. Безопасность и наблюдение**
VQA может быть полезен для анализа изображений в системах безопасности:
- **Видеоаналитика**: Ответы на вопросы типа «Сколько людей находится в кадре?» или «Что делает человек на изображении?».
- **Управление системами наблюдения**: Более точная обработка данных с камер для выявления событий или аномалий.

---

### **6. Образование и тренировка моделей**
VQA используется для:
- **Образовательных приложений**: Создание интерактивных материалов, которые могут отвечать на вопросы учащихся по изображениям, например, в биологии («Какая часть растения показана?») или географии («Какой это город?»).
- **Исследований ИИ**: Разработка новых методов мультимодальной обработки данных, таких как совмещение текстовых и визуальных признаков.

---

### **7. Медицинская визуализация**
Датасет может быть адаптирован для работы в медицинской сфере:
- **Диагностические системы**: Анализ медицинских изображений с ответами на вопросы врачей, например, «Есть ли опухоль на снимке?» или «Какие изменения заметны?».

---

### **8. Развлечения и игры**
Применение в создании интерактивных игр, где игрок может задавать вопросы о виртуальной сцене:
- **Игры с дополненной реальностью**: Взаимодействие с виртуальными объектами через вопросы и ответы.
- **Кино и медиа**: Анализ сцен в фильмах или сериалов для автоматической генерации описаний.

---

### **9. Автономные транспортные средства**
- **Навигация**: Ответы на вопросы типа «Что находится перед автомобилем?» или «Есть ли препятствия на дороге?».
- **Контекстное понимание окружающей среды**: Анализ ситуации на дороге для принятия решений.

---

### **10. Научные исследования**
- Анализ мультимодальных данных для изучения человеческого восприятия и понимания визуальной информации.
- Исследование сложности вопросов, связанных с визуальными задачами.

Датасет Visual Question Answering (VQA) обладает множеством преимуществ, которые делают его важным инструментом для исследований и приложений в области мультимодального искусственного интеллекта.

---

### **1. Мультимодальность**
- **Объединение текста и изображений**: VQA сочетает визуальные и текстовые данные, что позволяет моделям учиться работать с двумя типами информации одновременно.
- **Разнообразие задач**: Датасет подходит для задач распознавания изображений, обработки текста, семантического анализа и их объединения.

---

### **2. Реалистичность данных**
- **Естественные изображения**: Используются изображения из реального мира (например, из MS COCO), что делает задачи релевантными для реальных приложений.
- **Естественные вопросы**: Вопросы и ответы, собранные от людей, имеют большую естественность и разнообразие, чем автоматически сгенерированные данные.

---

### **3. Разнообразие данных**
- **Широкий спектр тем**: Включает вопросы о цветах, количестве объектов, действиях, взаимодействиях, местоположениях, текстурах и многом другом.
- **Разные уровни сложности**: Датасет охватывает как простые вопросы (например, «Какого цвета яблоко?»), так и сложные, требующие рассуждений (например, «Что произойдет дальше?»).

---

### **4. Поддержка обучения и исследований**
- **Универсальность**: Используется для обучения и тестирования мультимодальных моделей, таких как трансформеры или мультимодальные сетевые архитектуры.
- **Тестирование генерализации**: Проверяет способность моделей отвечать на ранее неизвестные вопросы, основанные на новых изображениях.
- **Обучение связей между объектами**: Модели могут изучать отношения между объектами, такие как близость, взаимодействие и взаимное расположение.

---

### **5. Воспроизводимость и стандартизация**
- **Стандартный набор данных**: VQA является общеизвестным и широко используемым в сообществе искусственного интеллекта, что позволяет легко сравнивать результаты различных моделей.
- **Четкие метрики оценки**: Метрики, такие как точность ответов, помогают объективно измерять производительность моделей.

---

### **6. Применимость в различных областях**
- **Многоцелевое использование**: Датасет может быть использован в обучении моделей для анализа изображений, создания описаний и ответов на вопросы.
- **Подготовка к реальным задачам**: Сценарии, заложенные в VQA, близки к реальным приложениям, таким как помощники с ИИ, медицинский анализ и системы наблюдения.

---

### **7. Простота использования**
- **Легкий доступ**: Датасет хорошо документирован и интегрирован в исследовательские фреймворки, такие как ParlAI и PyTorch.
- **Совместимость**: Поддерживает работу с современными инструментами машинного обучения, включая трансформеры, мультимодальные модели и библиотеки для работы с изображениями.

---

### **8. Вызовы для ИИ**
- **Поддержка новых подходов**: Датасет мотивирует исследователей разрабатывать архитектуры, объединяющие текст и изображения.
- **Сложные задачи**: Модели должны справляться с распознаванием объектов, логическим выводом и пониманием контекста, что поднимает планку развития ИИ.

---

### **9. Доступ к большому количеству данных**
- **Обширность датасета**: Включает сотни тысяч вопросов и изображений, что предоставляет богатый материал для обучения моделей.
- **Разнообразие форматов**: Вопросы представлены в текстовом формате, а изображения в виде реальных сцен, что позволяет изучать взаимодействие между ними.

---

### **10. Адаптивность**
- **Адаптация к другим задачам**: Хотя датасет разработан для VQA, его можно адаптировать для создания описаний изображений, генерации текстов или других мультимодальных задач.
- **Поддержка дообучения**: Модели, обученные на VQA, могут быть дообучены для более специализированных задач, таких как медицинская визуализация или интерактивные системы.

---

### **11. Стимулирование развития ИИ**
- **Синергия технологий**: Датасет способствует интеграции подходов из областей компьютерного зрения, обработки текста и логических рассуждений.
- **Тестирование новых идей**: Позволяет исследователям экспериментировать с различными архитектурами и подходами для мультимодальных задач.

---

### **12. Способствует улучшению пользовательского опыта**
- **Понимание естественного языка**: Разработка моделей, которые могут отвечать на вопросы, улучшает взаимодействие человека с ИИ.
- **Доступ к информации**: Системы, основанные на VQA, могут помогать пользователям интерпретировать изображения и предоставлять информацию в удобной форме.

---


Несмотря на широкие возможности, датасет Visual Question Answering (VQA) имеет ряд недостатков, которые следует учитывать при его использовании:

---

### **1. Ограниченность контекста**
- **Отсутствие дополнительной информации**: Датасет опирается только на изображение и текст вопроса, что ограничивает модели в доступе к внешним знаниям, например, о мире, событиях или культурных особенностях.
- **Недостаток мультимодального контекста**: Вопросы могут быть неоднозначными без дополнительного контекста, например, «Сколько здесь людей?» — если изображение частично обрезано, это вызывает проблемы интерпретации.

---

### **2. Качество аннотаций**
- **Ошибки и несоответствия в аннотациях**: Часть вопросов или ответов может быть некорректной или не согласованной, например, неправильное количество объектов или ошибки в грамматике.
- **Шум в данных**: Некоторые вопросы слишком общие или бессмысленные для анализа, такие как «Какого цвета это?» без явного объекта.
- **Субъективность ответов**: На некоторые вопросы могут быть несколько корректных ответов, что затрудняет обучение моделей (например, для вопроса «Как выглядит этот человек?» можно ответить разными способами).

---

### **3. Ограниченность данных**
- **Дисбаланс классов**: Частые ответы, такие как «да», «нет», «2», «красный», могут быть значительно переоценены в модели, а редкие ответы недостаточно представлены.
- **Неравномерность сложности**: Некоторые вопросы требуют простого извлечения признаков, а другие требуют сложного рассуждения, что создает разрыв в уровне сложности.
- **Ограниченность визуального содержания**: Датасет не охватывает все возможные визуальные сценарии, например, он плохо подходит для анализа медицинских изображений, научных данных или сложных графиков.

---

### **4. Узость лингвистической структуры**
- **Ограниченность языков**: Датасет в основном на английском языке, что затрудняет его использование в мультиязычных приложениях.
- **Повторяющиеся шаблоны вопросов**: Многие вопросы имеют одинаковую структуру, что упрощает задачу для моделей, но снижает их способность к генерализации на новые типы вопросов.

---

### **5. Отсутствие глубокой мультимодальности**
- **Ограниченность текстовой информации**: Отсутствие метаданных, таких как описание сцены, или связанного текста (например, текст, написанный на изображении), которые могли бы улучшить понимание.
- **Недостаточная связь с действиями**: Датасет больше ориентирован на статичные изображения, чем на динамические сцены, что ограничивает его применимость для анализа видео.

---

### **6. Проблемы с генерализацией**
- **Слабая адаптация к реальным данным**: Вопросы в реальной жизни могут быть более сложными и менее структурированными, чем те, что представлены в датасете.
- **Невозможность обработать неявные запросы**: Некоторые вопросы требуют логических выводов или внешнего знания, чего не предусмотрено в датасете.

---

### **7. Узкая направленность задач**
- **Преобладание тривиальных вопросов**: Многие вопросы относятся к простым задачам, таким как подсчет объектов или определение цвета, что не отражает сложных мультимодальных задач.
- **Мало вопросов высокого уровня**: Недостаточно задач, связанных с более сложным анализом, например, интерпретацией действий или связей между объектами.

---

### **8. Требования к вычислительным ресурсам**
- **Большие вычислительные затраты**: Работа с VQA требует значительных ресурсов для обработки изображений и текстов, что делает его недоступным для малых исследовательских групп или на слабом оборудовании.
- **Сложность мультимодальных моделей**: Совмещение обработки текстов и изображений увеличивает требования к архитектуре моделей и их обучению.

---

### **9. Ограниченная интерпретируемость**
- **Сложность анализа ошибок**: Трудно понять, почему модель дала неверный ответ — из-за ошибки в текстовой части, визуальной интерпретации или несогласованности данных.
- **Недостаток объяснений**: Датасет не предоставляет механизма для объяснения, почему был дан определенный ответ, что важно для доверия к системе.

---

### **10. Потенциальные этические проблемы**
- **Стереотипы в данных**: В датасете могут быть зафиксированы культурные или социальные стереотипы, которые обученные модели могут повторять.
- **Ошибки при использовании в критически важных областях**: Например, неправильные ответы в медицинской или юридической области могут привести к серьезным последствиям.

---